In [1]:
!pip install transformers[sentencepiece] transformers[torch] datasets evaluate sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 41.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 20.9 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
from transformers import pipeline, set_seed
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
import torch
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Teacher Model - Pegasus Large

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [5]:

# Define model and tokenizer. Here we use the PEGASUS model fine-tuned on XSum as an example.
model_ckpt = "google/pegasus-large"

tokenizer_lora = AutoTokenizer.from_pretrained(model_ckpt)

model_b = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

In [6]:
print(model_b)

PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(1024, 1024)
      (layers): ModuleList(
        (0-15): 16 x PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_no

In [6]:
from peft import LoraConfig, get_peft_model, TaskType

# Configure LoRA settings.
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,  # Task type for sequence-to-sequence language modeling.
    r=8,                            # Rank of the LoRA update matrices.
    lora_alpha=32,                  # Scaling factor.
    lora_dropout=0.1,               # Dropout probability for the LoRA layers.
    target_modules=["q_proj","k_proj", "v_proj","shared","out_proj","lm_head"]       # Typically, the attention query and value projection layers.
)

# Wrap the original model with LoRA adapters.
model_lora = get_peft_model(model_b, lora_config)
model_lora.print_trainable_parameters()  # Optional: Print the trainable parameters for verification.




trainable params: 4,699,760 || all params: 575,496,816 || trainable%: 0.8166


/usr/local/lib/python3.10/dist-packages/peft/tuners/tuners_utils.py:543: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [9]:
x = load_dataset("cnn_dailymail", "3.0.0")
x

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [10]:

dataset = x  # Replace with your dataset name if necessary

# Split the train set to retain only 1/4
train_subset = dataset["train"].train_test_split(test_size=0.1, seed=42)["test"]

# Create a new dataset dictionary
new_dataset = {
    "train": train_subset,
    "validation": dataset["validation"],
    "test": dataset["test"]
}

# Verify the new dataset sizes
print(f"Train: {len(new_dataset['train'])}")
print(f"Validation: {len(new_dataset['validation'])}")
print(f"Test: {len(new_dataset['test'])}")


Train: 28712
Validation: 13368
Test: 11490


In [13]:
# Defining the Preprocessing Function and Tokenizing the Input Dialogues:
def convert_examples_to_features(example_batch):
    input_encodings = t_tokenizer(example_batch['article'] , max_length = 1024, truncation = True )

# Tokenizing the Target Summaries:
    with t_tokenizer.as_target_tokenizer():
        target_encodings = t_tokenizer(example_batch['highlights'], max_length = 64, truncation = True )

# Returning the Tokenized Features:
    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

train = new_dataset['train'].map(convert_examples_to_features, batched = True)
test = new_dataset['test'].map(convert_examples_to_features, batched = True)
valid = new_dataset['validation'].map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/28712 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

In [10]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer_lora, model=model_lora)

In [18]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='teacher_model_lora',
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy='no',
    save_steps=500,
    gradient_accumulation_steps=16
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
import os
os.environ["WANDB_MODE"] = "disabled"

In [21]:
trainer = Trainer(model=model_lora, args=trainer_args,
                  tokenizer=tokenizer_lora, data_collator=seq2seq_data_collator,
                  train_dataset=train,
                  eval_dataset=test)
trainer.train()

<ipython-input-21-7de25705ea1d>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_lora, args=trainer_args,
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
10,2.876200
20,3.061400
30,2.829800
40,2.951400
50,2.932100
60,3.222000
70,2.785600
80,3.023200
90,2.865300
100,2.967900


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers fou

TrainOutput(global_step=1794, training_loss=2.058553509356053, metrics={'train_runtime': 7636.1577, 'train_samples_per_second': 3.76, 'train_steps_per_second': 0.235, 'total_flos': 6.152736813076339e+16, 'train_loss': 2.058553509356053, 'epoch': 0.9997213708553915})

In [22]:
model_lora.save_pretrained("./teacher_modelN")
tokenizer_lora.save_pretrained("./teacher_modelN")

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


('./teacher_modelN/tokenizer_config.json',
 './teacher_modelN/special_tokens_map.json',
 './teacher_modelN/spiece.model',
 './teacher_modelN/added_tokens.json',
 './teacher_modelN/tokenizer.json')

Folder successfully zipped as teacher_model.zip


In [23]:
!zip -r file2.zip /kaggle/working/teacher_modelN

  adding: kaggle/working/teacher_modelN/ (stored 0%)
  adding: kaggle/working/teacher_modelN/tokenizer.json (deflated 78%)
  adding: kaggle/working/teacher_modelN/adapter_config.json (deflated 55%)
  adding: kaggle/working/teacher_modelN/tokenizer_config.json (deflated 94%)
  adding: kaggle/working/teacher_modelN/adapter_model.safetensors (deflated 7%)
  adding: kaggle/working/teacher_modelN/special_tokens_map.json (deflated 82%)
  adding: kaggle/working/teacher_modelN/spiece.model (deflated 50%)
  adding: kaggle/working/teacher_modelN/README.md (deflated 66%)


In [24]:
from IPython.display import FileLink
FileLink(r'file2.zip')

/kaggle/working/file2.zip

In [ ]:
import shutil

# Define the folder to zip and the output file name
folder_path = '/kaggle/working/teacher_model_lora'
output_filename = 'teacher_model_checkpoints'

# Create a zip archive
shutil.make_archive(output_filename, 'zip', folder_path)

print(f"Folder successfully zipped as {output_filename}.zip")


inference

In [25]:
text_to_summarize = """
We propose a memory-efficient finetuning algorithm for large language models (LLMs) that supports finetuning LLMs with 65B parameters in 2/3/4-bit precision on as little as one 24GB GPU. Our method, modular low-rank adaptation (ModuLoRA), integrates any user-specified weight quantizer with finetuning via low-rank adapters (LoRAs). Our approach relies on a simple quantization-agnostic backward pass that adaptively materializes low-precision LLM weights from a custom black-box quantization module. This approach enables finetuning 2-bit and 3-bit LLMs for the first time---leveraging state-of-the-art 2-bit QuIP# quantization and 3-bit OPTQ quantization---outperforming finetuning that relies on less sophisticated 4-bit and 8-bit methods. In our experiments, ModuLoRA attains competitive performance on text classification, natural language infernece, and instruction following tasks using significantly less memory than existing approaches, and we also surpass the state-of-the-art ROUGE score on a popular summarization task. We release ModuLoRA together with a series of low-precision models as part of LLMTOOLS, a user-friendly library for quantizing, running, and finetuning LLMs on consumer GPUs.
"""

In [26]:
# Tokenize input and move tensors to the same device as the model
inputs = tokenizer_lora.encode("summarize: " + text_to_summarize, return_tensors="pt", max_length=512, truncation=True).to(device)

# Generate summary (pass inputs as a keyword argument)
summary_ids = model_lora.generate(input_ids=inputs, max_length=150, num_beams=4, early_stopping=True)

# Decode and print the summary
summary_text = tokenizer_lora.decode(summary_ids[0], skip_special_tokens=True)
print("Summarized Text:", summary_text)


Summarized Text: This approach enables finetuning 2-bit and 3-bit LLMs for the first time---leveraging state-of-the-art 2-bit QuIP# quantization and 3-bit OPTQ quantization---outperforming finetuning that relies on less sophisticated 4- and 8-bit methods.


Performance evaluation

In [69]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

In [ ]:
device

In [71]:
import evaluate

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=None,
                               column_text="article",
                               column_summary="highlights"):
    # Default to GPU if available, otherwise use CPU
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move the model to the correct device (GPU/CPU)
    model.to(device)

    # Splitting the dataset into batches
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    # Processing each batch
    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        # Tokenizing the articles:
        inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                           padding="max_length", return_tensors="pt")

        # Move inputs to the same device as the model
        inputs = {key: value.to(device) for key, value in inputs.items()}

        # Generating summaries:
        summaries = model.generate(input_ids=inputs["input_ids"],
                                   attention_mask=inputs["attention_mask"],
                                   length_penalty=0.8, num_beams=8, max_length=128)

        # Decode the generated summaries
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True)
                             for s in summaries]

        # Replace unwanted tokens (like empty spaces)
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

        # Updating the metric
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    # Compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [72]:
rouge_metric = evaluate.load("rouge")
score = calculate_metric_on_test_ds(
    valid, rouge_metric, trainer.model, tokenizer_lora, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)


NameError: name 'tokenizer_lora' is not defined

Load the model

In [4]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from peft import PeftModel

# Define paths
base_model_name = "google/pegasus-large"  # Replace with your base model
lora_checkpoint_path = "/kaggle/input/peg-large/kaggle/working/teacher_modelN"  # Replace with your LoRA checkpoint path

# Load the base Pegasus model and tokenizer
t_tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_name)

# Load the LoRA weights into the base model
teacher_model = PeftModel.from_pretrained(base_model, lora_checkpoint_path)

print("LoRA model loaded successfully!")


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/tuners/tuners_utils.py:543: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


LoRA model loaded successfully!


In [5]:
print(teacher_model)

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): PegasusForConditionalGeneration(
      (model): PegasusModel(
        (shared): lora.Embedding(
          (base_layer): Embedding(96103, 1024, padding_idx=0)
          (lora_dropout): ModuleDict(
            (default): Dropout(p=0.1, inplace=False)
          )
          (lora_A): ModuleDict()
          (lora_B): ModuleDict()
          (lora_embedding_A): ParameterDict(  (default): Parameter containing: [torch.FloatTensor of size 8x96103])
          (lora_embedding_B): ParameterDict(  (default): Parameter containing: [torch.FloatTensor of size 1024x8])
          (lora_magnitude_vector): ModuleDict()
        )
        (encoder): PegasusEncoder(
          (embed_tokens): Embedding(96103, 1024, padding_idx=0)
          (embed_positions): PegasusSinusoidalPositionalEmbedding(1024, 1024)
          (layers): ModuleList(
            (0-15): 16 x PegasusEncoderLayer(
              (self_attn): PegasusAttention(
                (k_pr

# Student Model

In [6]:
# Define model and tokenizer. Here we use the PEGASUS model fine-tuned on XSum as an example.
model_ckpt = "google/pegasus-x-base"

s_tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

tokenizer_config.json:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.60M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

In [7]:
from peft import LoraConfig, get_peft_model, TaskType

# Configure LoRA settings.
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,  # Task type for sequence-to-sequence language modeling.
    r=8,                            # Rank of the LoRA update matrices.
    lora_alpha=32,                  # Scaling factor.
    lora_dropout=0.1,               # Dropout probability for the LoRA layers.
    target_modules=["q_proj","k_proj", "v_proj","shared","out_proj","lm_head"]       # Typically, the attention query and value projection layers.
)

# Wrap the original model with LoRA adapters.
s_model = get_peft_model(model, lora_config)
s_model.print_trainable_parameters()  # Optional: Print the trainable parameters for verification.




trainable params: 3,319,408 || all params: 275,611,504 || trainable%: 1.2044


In [31]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


Teacher training - cross entropy and distillation loss

In [52]:
import torch

def shift_tokens_right(input_ids: torch.Tensor, pad_token_id: int, decoder_start_token_id: int) -> torch.Tensor:
    """
    Shift input ids one token to the right, and wrap the last token in the sequence as the first token in the output.
    This is required for decoder input preparation during training.
    """
    shifted_input_ids = input_ids.clone()
    shifted_input_ids[..., 1:] = input_ids[..., :-1].clone()
    shifted_input_ids[..., 0] = decoder_start_token_id

    if pad_token_id is not None:
        # Replace any -100s in the labels with the pad token ID to prevent loss errors
        shifted_input_ids.masked_fill_(shifted_input_ids == -100, pad_token_id)

    return shifted_input_ids


In [48]:
trainer_args = TrainingArguments(
    output_dir='baap',
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=100,
    evaluation_strategy='no',
    save_steps=500,
    gradient_accumulation_steps=16
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [59]:
import torch
from torch.nn import KLDivLoss, CrossEntropyLoss
from transformers import Trainer

class DistillationTrainer(Trainer):
    def __init__(self, teacher_model, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model

    def compute_loss(self, model, inputs, return_outputs=False,**kwargs):
        device = model.device

        # Move inputs to the same device as the model
        inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}

        labels = inputs["labels"]
        decoder_input_ids = shift_tokens_right(labels, model.config.pad_token_id, model.config.decoder_start_token_id)
        inputs["decoder_input_ids"] = decoder_input_ids

        # Forward pass on the student
        student_outputs = model(**inputs)

        # Forward pass on the teacher
        self.teacher_model.to(device)  # Ensure the teacher model is on the same device
        with torch.no_grad():
            teacher_outputs = self.teacher_model(**inputs)

        # Extract logits
        student_logits = student_outputs.logits
        teacher_logits = teacher_outputs.logits

        # Loss functions
        kl_loss_fn = KLDivLoss(reduction="batchmean")
        ce_loss_fn = CrossEntropyLoss()

        # KL Loss between teacher and student distributions
        kl_loss = kl_loss_fn(
            torch.log_softmax(student_logits / 2.0, dim=-1),
            torch.softmax(teacher_logits / 2.0, dim=-1)
        )

        # Hard label loss
        ce_loss = ce_loss_fn(student_logits.view(-1, student_logits.size(-1)), labels.view(-1))

        # Combine losses
        loss = 0.5 * kl_loss + 0.5 * ce_loss

        return (loss, student_outputs) if return_outputs else loss


In [49]:
import os
os.environ["WANDB_MODE"] = "disabled"

In [61]:
# Initialize the distillation trainer
trainer = DistillationTrainer(
    teacher_model=teacher_model,  # Pass the teacher model explicitly
    model=s_model,
    args=trainer_args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=t_tokenizer
)

trainer.train()

<ipython-input-59-15bb725e6ea5>:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `DistillationTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Step,Training Loss
100,52.952700
200,43.068300
300,29.518400
400,19.428100
500,14.667800
600,12.804900
700,12.031300
800,11.522000
900,11.252500
1000,11.069400


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers fou

TrainOutput(global_step=1794, training_loss=16.9464596479367, metrics={'train_runtime': 10180.3273, 'train_samples_per_second': 2.82, 'train_steps_per_second': 0.176, 'total_flos': 2.6127792598317696e+16, 'train_loss': 16.9464596479367, 'epoch': 0.9997213708553915})

In [62]:
s_model.save_pretrained("distilled_pegasus_base")
s_tokenizer.save_pretrained("distilled_pegasus_base")

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


('distilled_pegasus_base/tokenizer_config.json',
 'distilled_pegasus_base/special_tokens_map.json',
 'distilled_pegasus_base/spiece.model',
 'distilled_pegasus_base/added_tokens.json',
 'distilled_pegasus_base/tokenizer.json')

In [100]:
!zip -r baap.zip /kaggle/working/baap

  adding: kaggle/working/baap/ (stored 0%)
  adding: kaggle/working/baap/checkpoint-1794/ (stored 0%)
  adding: kaggle/working/baap/checkpoint-1794/adapter_config.json (deflated 55%)
  adding: kaggle/working/baap/checkpoint-1794/tokenizer_config.json (deflated 94%)
  adding: kaggle/working/baap/checkpoint-1794/training_args.bin (deflated 51%)
  adding: kaggle/working/baap/checkpoint-1794/scheduler.pt (deflated 56%)
  adding: kaggle/working/baap/checkpoint-1794/optimizer.pt (deflated 8%)
  adding: kaggle/working/baap/checkpoint-1794/trainer_state.json (deflated 69%)
  adding: kaggle/working/baap/checkpoint-1794/rng_state.pth (deflated 25%)
  adding: kaggle/working/baap/checkpoint-1794/special_tokens_map.json (deflated 82%)
  adding: kaggle/working/baap/checkpoint-1794/spiece.model (deflated 50%)
  adding: kaggle/working/baap/checkpoint-1794/adapter_model.safetensors (deflated 7%)
  adding: kaggle/working/baap/checkpoint-1794/tokenizer.json (deflated 78%)
  adding: kaggle/working/baap/ch

In [101]:
from IPython.display import FileLink
FileLink(r'baap.zip')

/kaggle/working/baap.zip

In [63]:
!zip -r distilled_pegasus.zip /kaggle/working/distilled_pegasus_base

  adding: kaggle/working/distilled_pegasus_base/ (stored 0%)
  adding: kaggle/working/distilled_pegasus_base/adapter_config.json (deflated 55%)
  adding: kaggle/working/distilled_pegasus_base/tokenizer_config.json (deflated 94%)
  adding: kaggle/working/distilled_pegasus_base/special_tokens_map.json (deflated 83%)
  adding: kaggle/working/distilled_pegasus_base/spiece.model (deflated 50%)
  adding: kaggle/working/distilled_pegasus_base/adapter_model.safetensors (deflated 7%)
  adding: kaggle/working/distilled_pegasus_base/tokenizer.json (deflated 78%)
  adding: kaggle/working/distilled_pegasus_base/README.md (deflated 66%)


In [64]:
from IPython.display import FileLink
FileLink(r'distilled_pegasus.zip')

/kaggle/working/distilled_pegasus.zip

inference

In [65]:
text_to_summarize = """
We propose a memory-efficient finetuning algorithm for large language models (LLMs) that supports finetuning LLMs with 65B parameters in 2/3/4-bit precision on as little as one 24GB GPU. Our method, modular low-rank adaptation (ModuLoRA), integrates any user-specified weight quantizer with finetuning via low-rank adapters (LoRAs). Our approach relies on a simple quantization-agnostic backward pass that adaptively materializes low-precision LLM weights from a custom black-box quantization module. This approach enables finetuning 2-bit and 3-bit LLMs for the first time---leveraging state-of-the-art 2-bit QuIP# quantization and 3-bit OPTQ quantization---outperforming finetuning that relies on less sophisticated 4-bit and 8-bit methods. In our experiments, ModuLoRA attains competitive performance on text classification, natural language infernece, and instruction following tasks using significantly less memory than existing approaches, and we also surpass the state-of-the-art ROUGE score on a popular summarization task. We release ModuLoRA together with a series of low-precision models as part of LLMTOOLS, a user-friendly library for quantizing, running, and finetuning LLMs on consumer GPUs.
"""

In [83]:
# Tokenize input and move tensors to the same device as the model
inputs = s_tokenizer.encode("summarize: " + valid[2]['article'] , return_tensors="pt", max_length=512, truncation=True).to(device)

# Generate summary (pass inputs as a keyword argument)
summary_ids = s_model.generate(input_ids=inputs, max_length=150, num_beams=4, early_stopping=True)

# Decode and print the summary
summary_text = s_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Summarized Text:", summary_text)


Summarized Text: Swansea's Bafetimbi Gomis has a history of fainting .


In [84]:
valid[2]['highlights']

'Bafetimbi Gomis collapses within 10 minutes of kickoff at Tottenham .\nBut he reportedly left the pitch conscious and wearing an oxygen mask .\nGomis later said that he was "feeling well"\nThe incident came three years after Fabrice Muamba collapsed at White Hart Lane .'

performance evaluation

In [85]:
import evaluate

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=None,
                               column_text="article",
                               column_summary="highlights"):
    # Default to GPU if available, otherwise use CPU
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move the model to the correct device (GPU/CPU)
    model.to(device)

    # Splitting the dataset into batches
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    # Processing each batch
    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        # Tokenizing the articles:
        inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                           padding="max_length", return_tensors="pt")

        # Move inputs to the same device as the model
        inputs = {key: value.to(device) for key, value in inputs.items()}

        # Generating summaries:
        summaries = model.generate(input_ids=inputs["input_ids"],
                                   attention_mask=inputs["attention_mask"],
                                   length_penalty=0.8, num_beams=8, max_length=128)

        # Decode the generated summaries
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True)
                             for s in summaries]

        # Replace unwanted tokens (like empty spaces)
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

        # Updating the metric
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    # Compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [88]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [96]:
y = x['validation'].select(range(0, len(x['validation']), 30))

In [97]:
y

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 446
})

In [98]:
rouge_metric = evaluate.load("rouge")
score = calculate_metric_on_test_ds(
    y, rouge_metric, s_model, s_tokenizer, batch_size = 1, column_text = 'article', column_summary= 'highlights'
)

100%|██████████| 446/446 [14:46<00:00,  1.99s/it]


In [99]:
score

{'rouge1': 0.02133428011528079,
 'rouge2': 0.0006971954129873818,
 'rougeL': 0.02093051883842114,
 'rougeLsum': 0.02126262873014098}

In [76]:
valid[0]['article']

'(CNN)Share, and your gift will be multiplied. That may sound like an esoteric adage, but when Zully Broussard selflessly decided to give one of her kidneys to a stranger, her generosity paired up with big data. It resulted in six patients receiving transplants. That surprised and wowed her. "I thought I was going to help this one person who I don\'t know, but the fact that so many people can have a life extension, that\'s pretty big," Broussard told CNN affiliate KGO. She may feel guided in her generosity by a higher power. "Thanks for all the support and prayers," a comment on a Facebook page in her name read. "I know this entire journey is much bigger than all of us. I also know I\'m just the messenger." CNN cannot verify the authenticity of the page. But the power that multiplied Broussard\'s gift was data processing of genetic profiles from donor-recipient pairs. It works on a simple swapping principle but takes it to a much higher level, according to California Pacific Medical Ce